In [71]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [72]:
os.chdir('/aichallenge')

In [73]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7fadf83f13c8>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7fade32e7c18>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/si

152.67506909370422


In [74]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0017349720001220703


In [76]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.5793604850769043


In [78]:
test.shape

(1888195, 237)

In [47]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

4.293514966964722


In [7]:
#remove columns which have any null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
#We would ignore all these 102 columns so we are left with 135 columns
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

46.40937829017639


In [8]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [9]:
len(unclean_columns)

36

In [10]:
len(clean_columns)

133

In [11]:
len(target_columns)

66

In [12]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.7251176834106445


In [13]:
# 56 messy columns are removed
len(messy_columns)

25

In [14]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
target_columns = train_missing.columns
t2=time.time()
print(t2-t1)

0.9124603271484375


In [15]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.6354084014892578


In [16]:
len(messy_columns)

61

In [17]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

1.9936518669128418


In [18]:
train.shape

(226140, 72)

In [19]:
train_missing.shape

(226140, 41)

In [20]:
#Remove the primary key column for data fitting
t1=time.time()
test_ids=test['innovation_challenge_key']
train = train.drop('innovation_challenge_key',axis=1)
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.2465553283691406


In [21]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.012174367904663086


In [22]:
scalcols

['contract_line_net_usd_amount', 'product_net_price']

In [23]:
catcols

['product_transaction_type',
 'contract_line_reaction_time_code',
 'sales_hierarchy_level',
 'service_sales_node_base_sales_hierarchy_level']

In [24]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.15871429443359375


In [25]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

0.6020236015319824


In [26]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price
count,2.261400e+05,2.261400e+05
mean,1.005455e-17,1.206546e-17
std,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01
25%,-1.252316e-01,-2.321716e-01
50%,-1.252316e-01,-2.321716e-01
75%,-1.252316e-01,-1.584198e-01
max,7.774224e+01,6.648806e+01


In [27]:
import gc
gc.collect()

74

In [28]:
t1=time.time()
train_objs_num = len(train)
dataset = pd.get_dummies(pd.concat(objs=[train, test], axis=0))
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

11.84855031967163


In [29]:
#check data quality now
dataset.isnull().any().any()

False

In [30]:
train.shape

(226140, 185)

In [31]:
test.shape

(1888195, 185)

In [34]:
def impute_nan(current_col):
    t1=time.time()
    train_test_X_index = train_missing[current_col].isnull()
    test_test_X_index = test_missing[current_col].isnull()
    train_X_index = train_missing[current_col].notnull()
    train_X = train[train_X_index]
    train_test_X = train[train_test_X_index]
    test_test_X = test[test_test_X_index]
    train_y = train_missing[current_col][train_X_index]
    train_test_y = train_missing[current_col][train_test_X_index]
    test_test_y = test_missing[current_col][test_test_X_index]
    X_train, X_valid, y_train, y_valid = train_test_split(train_X,train_y,test_size=0.33, random_state=42)
    neigh = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)
    neigh.fit(X_train,y_train)
    pred = neigh.predict(X_valid)
    acc = accuracy_score(pred,y_valid)
    print(current_col+':'+str(acc)+':'+str(time.time()-t1))
    pred_train =neigh.predict(train_test_X)
    print(time.time()-t1)
    pred_test = neigh.predict(test_test_X)
    print(time.time()-t1)
    t2=time.time()
    t=t2-t1
    return pred_train,pred_test,t,acc

In [35]:
accuracy=[]
time_taken=[]
for col in target_columns:
    train_y,test_y,time_,acc = impute_nan(col)
    accuracy.append(acc)
    time_taken.append(time_)
    train_test_X_index = train_missing[col].isnull()
    test_test_X_index = test_missing[col].isnull()
    train_missing[col][train_test_X_index]=train_y
    test_missing[col][test_test_X_index]=test_y
    gc.collect()

service_contract_process_type:0.9868785265623337:263.74833059310913
1329.9639189243317
7004.493754386902


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


product_bookings_measure_transaction_type:0.9407162022394436:440.9980511665344
707.4401526451111
1628.951546907425
product_adjustment_type:0.9407162022394436:455.47243547439575
762.8271799087524
1709.8757803440094
product_channel_bookings_yorn:0.9910247843518406:488.44204330444336
780.9261744022369
1967.2491106987
service_channel_bookings_yorn:0.9941640873205252:573.6198582649231
615.7729716300964
service_bookings_measure_transaction_type:0.7632092065613112:574.9788434505463
617.1755905151367
920.2483291625977
scms_name:0.999312170103324:482.56986355781555
734.0652973651886
1705.8272213935852
country_type:1.0:433.21938467025757
684.4211947917938
1664.5458834171295
service_product_base_product_family_owner:1.0:315.519344329834
739.5633130073547
4072.774183034897
service_product_base_service_brand_code:0.9810621323233137:618.2608788013458
625.5498886108398
645.8523187637329
status_code:0.9806645794982733:624.5886693000793
630.8739910125732
720.9757895469666
partner_country_registered_typ

In [36]:
print(train_missing.isnull().any().any())
print(test_missing.isnull().any().any())

False
False


In [37]:
train_missing.to_csv('train_imputed.csv')
test_missing.to_csv('test_imputed.csv')

In [50]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):
    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
        'booster' : 'gbtree',
        'max_depth' : int(max_depth),
        'gamma' : gamma,
        'eta' : 0.1,
        'objective' : 'binary:logistic',
        'nthread' : 4,
        'silent' : True,
        'eval_metric': 'logloss',
        'subsample' : max(min(subsample, 1), 0),
        'colsample_bytree' : max(min(colsample_bytree, 1), 0),
        'min_child_weight' : min_child_weight,
        'max_delta_step' : int(max_delta_step),
        'seed' : 1001
        }

    folds = 5
    cv_score = 0

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 500,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 20,
                    metrics = 'logloss',
                    show_stdv = True
               )

    val_score = xgbc['test-logloss-mean'].iloc[-1]
    train_score = xgbc['train-logloss-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)
    return (val_score*2) - 1

In [53]:
train_objs_num = len(train_missing)
dataset = pd.concat(objs=[train_missing, test_missing], axis=0)
dataset = pd.get_dummies(dataset)
train_m = dataset[:train_objs_num]
test_m = dataset[train_objs_num:]

In [59]:
X_train, X_valid, y_train, y_valid = train_test_split(train_m,train_y,test_size=0.33, random_state=42)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7fadf83f14e0>>
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [60]:
y_train.head()

227682    Y
120974    N
48142     Y
36978     N
124715    N
Name: renewed_yorn, dtype: object

In [61]:
import xgboost as xgb
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full1.log', 'a')
AUCbest = -1.
ITERbest = 0
y_dtrain=pd.get_dummies(y_train)
dtrain = xgb.DMatrix(X_train, label = y_dtrain['Y'])

In [62]:
from bayes_opt import BayesianOptimization
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 50.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [63]:
XGB_BO.explore({
              'max_depth':            [3, 8, 3, 8, 8, 3, 8, 3,5,3],
              'gamma':                [0.5, 8, 0.2, 9, 0.5, 8, 0.2, 9,25,50],
              'min_child_weight':     [0.2, 0.2, 0.2, 0.2, 12, 12, 12, 12,5,8],
              'max_delta_step':       [1, 2, 2, 1, 2, 1, 1, 2,1,2],
              'subsample':            [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8,0.8,0.8],
              'colsample_bytree':     [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8,0.8,0.8],
              })

In [64]:
t1=time.time()
import warnings
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)
t2=time.time()
print(t2-t1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 500 iterations with train-auc = 0.525818 val-auc = 0.531186 ( diff = -0.005368 ) train-gini = 0.051637 val-gini = 0.062372
    1 | 02m33s |    0.06237 |             0.6000 |    0.5000 |           1.0000 |      3.0000 |             0.2000 |      0.6000 | 
 Stopped after 500 iterations with train-auc = 0.404149 val-auc = 0.431698 ( diff = -0.027548 ) train-gini = -0.191702 val-gini = -0.136605
    2 | 06m32s |   -0.13660 |             0.8000 |    8.0000 |           2.0000 |      8.0000 |             0.2000 |      0.8000 | 
 Stopped after 500 iterations with train-auc = 0.

In [67]:
t1=time.time()
xgb1=xgb.XGBClassifier(max_depth=2,gamma=50,colsample_bytree=0.4,
                       eval_metric='logloss',max_delta_step= 10.0,min_child_weight= 0.0,
                      subsample= 0.999999999996573)
pred=xgb1.fit(X_train,y_train).predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
t2-t1

0.5832403342407773


12.724536180496216

In [66]:
XGB_BO.res['max']['max_params']

{'colsample_bytree': 0.4,
 'gamma': 50.0,
 'max_delta_step': 10.0,
 'max_depth': 2.0,
 'min_child_weight': 0.0,
 'subsample': 0.999999999996573}

In [70]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test_m)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-knn.csv',index=False)